In [1]:
import gc
import itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
items = pd.read_csv("/storage/k8s/my_kaggle/data_dir/competitive-data-science-predict-future-sales/items.csv")
shops = pd.read_csv("/storage/k8s/my_kaggle/data_dir/competitive-data-science-predict-future-sales/shops.csv")
train = pd.read_csv("/storage/k8s/my_kaggle/data_dir/competitive-data-science-predict-future-sales/sales_train.csv")
test = pd.read_csv("/storage/k8s/my_kaggle/data_dir/competitive-data-science-predict-future-sales/test.csv")

In [5]:
print(items.shape)
print(items.head(2))
print(shops.shape)
print(items.head(2))
print(train.shape)
print(items.head(2))
print(test.shape)
print(items.head(2))

(22170, 3)
                                           item_name  item_id  \
0          ! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.)         D        0   
1  !ABBYY FineReader 12 Professional Edition Full...        1   

   item_category_id  
0                40  
1                76  
(60, 2)
                                           item_name  item_id  \
0          ! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.)         D        0   
1  !ABBYY FineReader 12 Professional Edition Full...        1   

   item_category_id  
0                40  
1                76  
(2935849, 6)
                                           item_name  item_id  \
0          ! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.)         D        0   
1  !ABBYY FineReader 12 Professional Edition Full...        1   

   item_category_id  
0                40  
1                76  
(214200, 3)
                                           item_name  item_id  \
0          ! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.)         D        0   
1  !ABBYY FineReader 12 Professional Edi

In [18]:
train.describe()

,date_block_num,shop_id,item_id,item_price,item_cnt_day
count,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06
mean,1.456991e+01,3.300173e+01,1.019723e+04,8.908532e+02,1.242641e+00
std,9.422988e+00,1.622697e+01,6.324297e+03,1.729800e+03,2.618834e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-2.200000e+01
25%,7.000000e+00,2.200000e+01,4.476000e+03,2.490000e+02,1.000000e+00
50%,1.400000e+01,3.100000e+01,9.343000e+03,3.990000e+02,1.000000e+00
75%,2.300000e+01,4.700000e+01,1.568400e+04,9.990000e+02,1.000000e+00
max,3.300000e+01,5.900000e+01,2.216900e+04,3.079800e+05,2.169000e+03


In [19]:
train.head(2)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.0,1.0
1,03.01.2013,0,25,2552,899.0,1.0


In [25]:
train["date"] = pd.to_datetime(train["date"], format="%d.%m.%Y")
train["date"].unique()

array(['2013-01-02T00:00:00.000000000', '2013-01-03T00:00:00.000000000',
       '2013-01-05T00:00:00.000000000', ...,
       '2015-10-28T00:00:00.000000000', '2015-10-25T00:00:00.000000000',
       '2015-10-13T00:00:00.000000000'], dtype='datetime64[ns]')

In [29]:
train["shop_id"].unique()

array([59, 25, 24, 23, 19, 22, 18, 21, 28, 27, 29, 26,  4,  6,  2,  3,  7,
        0,  1, 16, 15,  8, 10, 14, 13, 12, 53, 31, 30, 32, 35, 56, 54, 47,
       50, 42, 43, 52, 51, 41, 38, 44, 37, 46, 45,  5, 57, 58, 55, 17,  9,
       49, 39, 40, 48, 34, 33, 20, 11, 36])

In [32]:
print(train[train["shop_id"]==57].head(2))
print(train[train["shop_id"]==0].head(2))

             date  date_block_num  shop_id  item_id  item_price  item_cnt_day
259653 2013-03-09               2       57     4164       999.0           1.0
259654 2013-03-06               2       57     4164       999.0           1.0
            date  date_block_num  shop_id  item_id  item_price  item_cnt_day
37589 2013-01-12               0        0    11059        73.0           1.0
37590 2013-01-22               0        0    11059        73.0           1.0


In [33]:
# Merge some duplicate shops
# train["shop_id"] = train["shop_id"].replace({0: 57, 1: 58, 11: 10, 40: 39})
# Keep only shops that are in the test set
train = train.loc[train.shop_id.isin(test["shop_id"].unique()), :]
# Drop training items with extreme or negative prices or sales counts
train = train[(train["item_price"] > 0) & (train["item_price"] < 50000)]
train = train[(train["item_cnt_day"] > 0) & (train["item_cnt_day"] < 1000)]